# Modelo markoviano de máxima entropía

Utilizamos un corpus en español y conllu para leer el corpus

In [1]:
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
fatal: destination path 'UD_Spanish-AnCora' already exists and is not an empty directory.


In [4]:
#leemos los datos que vienen del archivo conllu
from conllu import parse_incr
wordlist = []#creo una lista vacía


# Cálculo de conteos

En el modelo markoviano de máxima entropía calculamos las probabilidades de la siguiente forma:

$$
P(t_i|w_i, t_{i-1})=\frac{C(w_i,t_i,t_{i-1})}{C(w_i,t_{i-1})}=\frac{C(t_i|w_i|t_{i-1})}{C(w_i,t_{i-1})}
$$
(no está bien $C(t_i|w_i|t_{i-1})$ pero uso esta notacion)





In [46]:
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"

#tenemos 2 diccionarios inicialmente vacíos
upperCountDict = {} #la cuenta superior de la expresión
lowerCountDict = {} #la cuenta inferior de la expresión
token_anterior= {} #diccionario auxiliar para saber la expresión i-1

#Empieza a hacerse las cuentas
for tokenlist in parse_incr(data_file):
    cuenta = 0
    token_anterior['upos'] = 'None'
    for token in tokenlist:
        token_actual = token
        
        superior = token_actual['upos']+'|'+token_actual['form'].lower()+'|'+token_anterior['upos']
        inferior = token_actual['form'].lower()+'|'+token_anterior['upos']
            
        if superior in upperCountDict.keys():
            upperCountDict[superior] +=1
        else:
            upperCountDict[superior] = 1

        if inferior in lowerCountDict.keys():
            lowerCountDict[inferior] +=1
        else:
            lowerCountDict[inferior] = 1  

        token_anterior = token_actual

upperCountDict
             


{'DET|el|None': 281,
 'NOUN|gobernante|DET': 1,
 'PUNCT|,|NOUN': 971,
 'ADP|con|PUNCT': 77,
 'ADJ|ganada|ADP': 1,
 'NOUN|fama|ADJ': 1,
 'ADP|desde|NOUN': 17,
 'SCONJ|que|ADP': 145,
 'VERB|llegó|SCONJ': 1,
 'VERB|hace|VERB': 5,
 'NUM|16|VERB': 2,
 'NOUN|meses|NUM': 14,
 'ADP|al|NOUN': 81,
 'NOUN|poder|ADP': 8,
 'ADP|de|NOUN': 2029,
 'VERB|explotar|ADP': 1,
 'ADP|al|VERB': 113,
 'NOUN|máximo|ADP': 1,
 'DET|su|NOUN': 11,
 'NOUN|oratoria|DET': 1,
 'CCONJ|y|NOUN': 415,
 'ADJ|acusado|CCONJ': 2,
 'ADP|por|ADJ': 120,
 'DET|sus|ADP': 75,
 'NOUN|detractores|DET': 3,
 'NOUN|incontinencia|ADP': 1,
 'ADJ|verbal|NOUN': 2,
 'PUNCT|,|ADJ': 366,
 'VERB|enmudeció|PUNCT': 1,
 'ADP|desde|VERB': 16,
 'DET|el|ADP': 555,
 'NOUN|momento|DET': 19,
 'ADP|en|NOUN': 304,
 'PRON|que|DET': 95,
 'DET|el|PRON': 31,
 'PROPN|tribunal|DET': 2,
 'PROPN|supremo|PROPN': 2,
 'ADP|de|PROPN': 269,
 'PROPN|justicia|ADP': 5,
 'PUNCT|(|PROPN': 136,
 'PROPN|tsj|PUNCT': 1,
 'PUNCT|)|PROPN': 130,
 'VERB|decidió|PUNCT': 4,
 'VERB|su

# Cálculo de probabilidades


In [47]:
probDict = {} # matriz A

#  Probabilities 
for key in upperCountDict.keys():
    tag, word, prevtag = key.split('|')
    consulta_inferior = word+'|'+prevtag

    if lowerCountDict[consulta_inferior]>0:
        probDict[key] =upperCountDict[key]/lowerCountDict[consulta_inferior]
    else:
        print(key)

# Cálculo del algoritmo de Viterbi


El cálculo de probabilidades de Viterbi para una categoría gramatical $j$ en una columna $t$ es
$$
v_j(t)=\max{v_{t-1}}(i)\times P(j|palabra,i,...)
$$



In [48]:
#identificamos las categorías gramaticales 'upos' únicas en el corpus

stateSet = set([w.split('|')[0] for w in list(upperCountDict.keys())])

stateSet

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}

In [49]:
#A cada categoría asignamos un índice entero 

tagStateDict = {}
for i, state in enumerate(stateSet):
    tagStateDict[state] = i

tagStateDict

{'INTJ': 0,
 'VERB': 1,
 'PART': 2,
 'NOUN': 3,
 'SCONJ': 4,
 'SYM': 5,
 '_': 6,
 'DET': 7,
 'AUX': 8,
 'ADJ': 9,
 'PRON': 10,
 'PROPN': 11,
 'ADV': 12,
 'ADP': 13,
 'NUM': 14,
 'CCONJ': 15,
 'PUNCT': 16}

In [50]:
#Calculamos distribución inicial de estados
wordlist = []#creo una lista vacía
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"

count = 0 # contador de la longitud del corpus

initTagStateProb = {} #\rho_i^0
for tokenlist in parse_incr(data_file):
    count += 1
    tag = tokenlist[0]['upos']
    if tag in initTagStateProb.keys():
        initTagStateProb[tag] +=1
    else:
        initTagStateProb[tag]=1

for key in initTagStateProb.keys():
    initTagStateProb[key] /= count

initTagStateProb

{'DET': 0.3633615477629988,
 'PROPN': 0.1124546553808948,
 'ADP': 0.16384522370012092,
 'PRON': 0.034461910519951636,
 'SCONJ': 0.02418379685610641,
 'ADV': 0.06287787182587666,
 'PUNCT': 0.07799274486094317,
 'VERB': 0.04353083434099154,
 'ADJ': 0.010882708585247884,
 'CCONJ': 0.03325272067714631,
 'NOUN': 0.02720677146311971,
 '_': 0.0006045949214026602,
 'INTJ': 0.0006045949214026602,
 'AUX': 0.022370012091898428,
 'NUM': 0.01995163240628779,
 'SYM': 0.0006045949214026602,
 'PART': 0.0018137847642079807}

In [54]:
import nltk
import numpy as np
nltk.download('punkt')
from nltk import word_tokenize

def Viterbitags(secuencia, probDict = probDict, tagStateDict = tagStateDict, initTagStateProb=initTagStateProb):
    #inicialización de la primera columna

    seq = word_tokenize(secuencia)
    viterbiProb = np.zeros((17, len(seq)))

    # para la primera columna
    for key in tagStateDict.keys():
        tag_row = tagStateDict[key] #en tag_row está el número del cada categoría gramatical
        word_tag= key+'|'+seq[0].lower()+'|'+'None'
        if word_tag in probDict.keys():
            viterbiProb[tag_row, 0 ] = initTagStateProb[key]*probDict[word_tag]
    

    # para las siguientes columnas
    for col in range(1, len(seq)):
        for key in tagStateDict.keys():
            tag_row = tagStateDict[key]  #en tag_row está el número del cada categoría gramatical
            
            possible_probs = []
            for key2 in tagStateDict.keys():
                tag_row2 = tagStateDict[key2]  #voy a variar i=tag_row2 para hallar el max_i

                word_tag= key+'|'+seq[col].lower()+'|'+key2
                #print(word_tag)

                if word_tag in probDict.keys():
                    possible_probs.append(viterbiProb[tag_row2, col-1]*probDict[word_tag]) #calculo la probabilidad de Viterbi
                
            viterbiProb[tag_row, col] = max(possible_probs, default=0)
    
    #construccion de la sección de tags
    res= []
    for i, p in enumerate(seq):
        for tag in tagStateDict.keys():
            if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
                res.append((p,tag))

    return res

matrix = Viterbitags('estos instrumentos han de rasgar')
matrix


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('estos', 'DET'),
 ('instrumentos', 'NOUN'),
 ('han', 'AUX'),
 ('de', 'ADP'),
 ('rasgar', 'VERB')]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0ff8abcd-21e6-41c5-ad5e-88d416b585fc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>